In [1]:
from dl import authClient as ac
from dl import storeClient as sc
from dl import queryClient as qc
import pandas as pd
import numpy as np
from dl.helpers.utils import convert        # to use Pandas
import os, getpass

do_cleanup = True                          # Remove any files/tables created by this notebook

token = ac.login(input('Enter user name (+ENTER): '),getpass.getpass('Enter password (+ENTER): '))

print('User: ' + ac.whoAmI())

Enter user name (+ENTER):  i_am_your_father
Enter password (+ENTER):  ········


User: i_am_your_father


In [16]:
print('User: ' + ac.whoAmI())

User: i_am_your_father


In [17]:
# std lib
from astropy.table import Table

# 3rd party
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from astropy.utils.data import download_file  #import file from URL
%matplotlib inline

# Data Lab
from dl import authClient as ac, queryClient as qc, helpers
print('Done importing')

Done importing


In [18]:
catalog = pd.read_csv('consolidated_sources_with_class.csv')
catalog

,ra,dec,references,n_spectra,in_DESI_DR1,TARGETID,Classification,Comments,Sure
0,268.743723,18.901710,H22,0,False,NaN,NaN,NaN,NaN
1,163.698957,36.572058,H22,0,False,NaN,NaN,NaN,NaN
2,179.093353,21.987776,H22,1,True,39628306840030402,lqso/new,NaN,NaN
3,74.154194,-63.635577,H22,0,False,NaN,NaN,NaN,NaN
4,218.948869,33.886035,H22,2,True,39632961426949595/39632961426949605,qso+gal,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1719,227.100970,-0.101002,H25,2,True,2780954867269635/39627788537301839,NaN,NaN,NaN
1720,359.073255,-31.951281,H25,0,False,NaN,NaN,NaN,NaN
1721,359.071670,-35.537485,H25,0,False,NaN,NaN,NaN,NaN
1722,359.265790,-34.377480,H25,0,False,NaN,NaN,NaN,NaN


In [19]:
# 条件 1：reference列是"H22", "D22;H22", 或 "D22"
ref_ok = catalog["references"].isin(["H22", "D22;H22", "D22"])

# 条件 2：Classification列是"lqso/new"
class_ok = catalog["Classification"] == "lqso/new"

# 筛选满足两个条件的行
selected = catalog[ref_ok & class_ok]

In [20]:
# selected.to_csv("fh2_input_selected.csv")

In [48]:
pd.read_csv("fh2_input_selected.csv")

,ra,dec,references
0,179.093353,21.987776,H22
1,244.706042,6.476774,D22;H22
2,100.652938,56.286423,D22;H22
3,4.946581,1.500982,H22
4,65.581298,-4.797160,D22;H22
5,9.316034,20.973797,D22;H22
6,234.254361,32.390780,D22;H22
7,246.258188,43.158860,H22
8,357.042088,15.505757,D22;H22
9,217.230942,5.005704,D22;H22


In [49]:
print ('Local file result: '   + qc.mydb_import('fh2', 'fh2_input_selected.csv'))

Local file result: OK


In [50]:
# ls9
# 下面是通过调用在线数据库的 all_catalog_galhighz 与 datalab 的 ls_dr9.tractor 进行匹配，q3c_join(o.ra, o.dec, t.ra, t.dec, 0.001) 半径为 0.001 degree，LIMIT 1 选取最近的一个
# 然后通过 tr.ls_id = ap.ls_id 获取 ls_dr9.apflux 相同源的孔径通量，如果不需要就可以直接 去掉ls_dr9.apflux AS ap 以及相应的 ap.
# https://datalab.noirlab.edu/query.php?name=ls_dr9.tractor 是 ls_dr9.tractor 的星表信息
sql = '''SELECT o.*, tr.ls_id,tr.type, tr.ra as ra_out, tr.dec as dec_out,
                tr.dered_mag_g as mag_g, tr.dered_mag_r as mag_r, tr.dered_mag_i as mag_i, tr.dered_mag_z as mag_z, 
                tr.dered_mag_w1 as mag_w1, tr.dered_mag_w2 as mag_w2,
                tr.dered_flux_g as flux_g, tr.dered_flux_r as flux_r,tr.dered_flux_z as flux_z, 
                tr.dered_flux_w1 as flux_w1, tr.dered_flux_w2 as flux_w2,
                ap.apflux_g_1, ap.apflux_g_2, ap.apflux_g_3, ap.apflux_g_4, ap.apflux_g_5, ap.apflux_g_6, ap.apflux_g_7, ap.apflux_g_8,
                ap.apflux_r_1, ap.apflux_r_2, ap.apflux_r_3, ap.apflux_r_4, ap.apflux_r_5, ap.apflux_r_6, ap.apflux_r_7, ap.apflux_r_8,
                ap.apflux_z_1, ap.apflux_z_2, ap.apflux_z_3, ap.apflux_z_4, ap.apflux_z_5, ap.apflux_z_6, ap.apflux_z_7, ap.apflux_z_8,
                ap.apflux_w1_1, ap.apflux_w1_2, ap.apflux_w1_3, ap.apflux_w1_4, ap.apflux_w1_5,
                ap.apflux_w2_1, ap.apflux_w2_2, ap.apflux_w2_3, ap.apflux_w2_4, ap.apflux_w2_5
         FROM mydb://fh2 AS o,
              ls_dr10.apflux AS ap
         LEFT JOIN LATERAL (
               SELECT t.* 
                    FROM 
                        ls_dr10.tractor AS t
                    WHERE
                        q3c_join(o.ra, o.dec, t.ra, t.dec, 0.001)
                    ORDER BY
                        q3c_dist(o.ra, o.dec, t.ra, t.dec)
                    ASC LIMIT 3
               ) as tr ON true
        WHERE
             tr.ls_id = ap.ls_id
               ;
               '''

df = qc.query(sql=sql,fmt='pandas',timeout=1200)

# df = df.drop_duplicates(subset='ls_id', keep='first')
df = df.reset_index(drop=True)

df

,ra,dec,references,ls_id,type,ra_out,dec_out,mag_g,mag_r,mag_i,...,apflux_w1_1,apflux_w1_2,apflux_w1_3,apflux_w1_4,apflux_w1_5,apflux_w2_1,apflux_w2_2,apflux_w2_3,apflux_w2_4,apflux_w2_5
0,179.093353,21.987776,H22,10995593221508804,PSF,179.093347,21.987776,22.705090,22.668655,22.840822,...,31.955244,66.857380,94.740160,110.465320,121.663950,19.259684,40.705700,58.521805,69.253780,77.070015
1,179.093353,21.987776,H22,10995593221508815,REX,179.093882,21.987996,20.438988,19.533415,18.829592,...,36.627900,73.537610,97.646710,112.408330,122.752290,22.301937,45.456764,61.342377,71.420135,77.887920
2,179.093353,21.987776,H22,10995593221508813,PSF,179.093790,21.988411,23.064306,22.923447,22.960268,...,34.649840,69.742130,94.926704,110.967460,121.387140,21.258104,43.136986,59.371240,70.062675,76.893875
3,244.706042,6.476774,D22;H22,10995502918144465,PSF,244.705929,6.476708,22.008064,21.421616,20.872130,...,23.099176,45.943577,58.378246,65.482900,74.200134,26.528307,53.207962,68.850586,76.447556,83.672150
4,244.706042,6.476774,D22;H22,10995502918144481,SER,244.706237,6.476812,20.048891,19.544216,19.372963,...,24.338629,45.952160,58.429447,65.293594,74.443530,27.865042,53.029427,68.617470,76.879290,84.145836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,183.031900,28.440900,D22,10995628786061965,REX,183.031728,28.440800,20.071552,19.592497,19.187643,...,38.914330,77.941185,97.001870,105.785960,112.267030,48.485855,99.297420,126.063080,137.044660,142.027740
64,183.031900,28.440900,D22,10995628786061971,PSF,183.032056,28.441020,18.485168,18.312317,18.076078,...,40.782715,78.561806,96.942070,106.020580,112.253500,51.329033,100.343450,126.259570,137.495450,141.995960
65,183.031900,28.440900,D22,10995628786061946,EXP,183.031068,28.440975,24.672760,22.560850,22.128262,...,28.882442,64.370540,92.211266,104.848940,113.071910,36.025850,81.661000,117.581220,133.290280,141.092650
66,232.943400,37.402900,D22,9907734172600498,PSF,232.943426,37.403022,19.891373,19.693870,-9999.000000,...,36.457085,66.644010,90.702510,109.722404,129.433980,42.154907,76.920494,103.131160,120.145160,134.571660


In [51]:
filename = 'fh2_desils.csv'
if os.path.exists(filename):
    os.remove(filename)

df.to_csv(filename,index=None)
